In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import torch
from torch import optim
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR100
from torchvision.transforms import v2
from pathlib import Path
import wandb
import os
import sys

sys.path.append("../src")

from trainer import Trainer
from module import ResNetModule
from utils import model_size, load_from_checkpoint
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset
import settings as s

In [3]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [4]:
logger = WandbLogger(
    project_name=s.project_name,
    config={
        "model": s.model,
        "dataset": s.dataset,
        "max_epochs": s.max_epochs,
        "optimizer": s.optimizer,
        "lr_scheduler": s.lr_scheduler
    },
    logs_path=logs_path,
    offline=s.wandb_offline
)

In [5]:
# cpu_count = os.cpu_count()
cpu_count = 7

dataset = CIFAR100(data_path, train=True, download=True)

train_dataset, val_dataset = random_split(
    dataset, [s.dataset["train_split"], s.dataset["val_split"]]
)

normalize_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

transforms_list = []
if s.dataset["augumentations"]:
    transforms_list.extend([
        v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
        v2.RandomHorizontalFlip(),
        
        # v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),  # Random cropping
        # v2.RandomHorizontalFlip(),  # Horizontal flip
        # v2.RandomVerticalFlip(p=0.2),  # Vertical flip with 20% probability
        # v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color adjustments
        # v2.RandomRotation(degrees=15),  # Random rotation within ±15 degrees
        # v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
        # v2.RandomGrayscale(p=0.1),  # Convert to grayscale with 10% probability
    ])

# Add normalization (always)
transforms_list.append(normalize_transforms)

# Compose the transforms
train_transforms = v2.Compose(transforms_list)
val_transforms = normalize_transforms

train_dataset = MapDataset(train_dataset, transform=train_transforms)
val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=s.dataset["batch_size"], shuffle=True, num_workers=cpu_count, pin_memory=True)
val_dataloader = DataLoader(
    val_dataset, batch_size=s.dataset["batch_size"],  num_workers=cpu_count, pin_memory=True)

Files already downloaded and verified


In [6]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=1, batch_size=10, max_epochs=500, augument_data=False)
]

In [7]:
module = ResNetModule(toy_model=False)
optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=s.optimizer["weight_decay"] if s.optimizer["weight_decay"] else 0.01
)

try:
    if s.lr_scheduler["name"] == "OneCycleLR":
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=s.lr_scheduler["max_lr"],
            epochs=s.max_epochs,
            steps_per_epoch=len(train_dataloader),
        )

        print(s.lr_scheduler["name"])
except TypeError:
    lr_scheduler = None
    print("lr_scheduler is None!")

OneCycleLR


In [8]:
# module.model, optimizer = load_from_checkpoint(
#     path="../logs/wandb/offline-run-20241215_132918-77n093vj/checkpoints/best.pt",
#     model=module.model,
#     optimizer=optimizer
# )

summary(module.model, input_size=(train_dataset[0][0].shape), batch_size=s.dataset["batch_size"], device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [32, 64, 32, 32]           1,728
       BatchNorm2d-2           [32, 64, 32, 32]             128
              ReLU-3           [32, 64, 32, 32]               0
         ConvBlock-4           [32, 64, 32, 32]               0
            Conv2d-5          [32, 128, 32, 32]          73,728
       BatchNorm2d-6          [32, 128, 32, 32]             256
              ReLU-7          [32, 128, 32, 32]               0
         ConvBlock-8          [32, 128, 32, 32]               0
            Conv2d-9          [32, 128, 32, 32]         147,456
      BatchNorm2d-10          [32, 128, 32, 32]             256
             ReLU-11          [32, 128, 32, 32]               0
           Conv2d-12          [32, 128, 32, 32]         147,456
      BatchNorm2d-13          [32, 128, 32, 32]             256
             ReLU-14          [32, 128,

In [9]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=s.fast_dev_run,
    measure_time=True,
    lr_scheduler=lr_scheduler,
    save_checkpoint_type="best_val",
    num_workers=cpu_count
)

Using device: cuda!


In [10]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


Run stopped!


lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step_train_accuracy,▁▁▁▁▁▁▁▅▅▁▁▅▅▅▁▅█▁▁▁▁▅██▁▁▅▁▁▁▁▁▅▁▁▅▁▅▁▅
step_train_loss,▄▄▄▁▄▃▃▃▃▃█▂▂▁▃▃▂▂▂▁▃▃▂▂▂▂▂▁▃▁▃▃▂▂▂▂▂▁▃▂
training_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇███
lr,0.0004
model_architecture,ResNet56( (feature...
step_train_accuracy,3.125
step_train_loss,4.7302
training_step,80


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)